In [2]:
!pip install weaviate-client langchain-community unstructured

from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Weaviate
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain import HuggingFaceHub
import weaviate
import os

In [ ]:
WEAVIATE_CLUSTER = "https://kxflyifistegeibyl6b8na.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = ""


!pip install "weaviate-client>=3.26.7,<4.0.0"
WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.Client(
    WEAVIATE_URL,
    weaviate.AuthApiKey(WEAVIATE_API_KEY)
)


In [4]:
# Extracting Text from Flask Documentation
urls = ["https://flask.palletsprojects.com/en/stable/"]  # Add more links if needed
loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': 'https://flask.palletsprojects.com/en/stable/'}, page_content='Navigation\n\nindex\n\nmodules |\n\nnext |\n\nFlask Documentation (3.1.x) »\n\nWelcome to Flask\n\nWelcome to Flask¶\n\n_images/flask-horizontal.png\n\nWelcome to Flask’s documentation. Flask is a lightweight WSGI web application framework. It is designed to make getting started quick and easy, with the ability to scale up to complex applications.\n\nGet started with Installation and then get an overview with the Quickstart. There is also a more detailed Tutorial that shows how to create a small but complete application with Flask. Common patterns are described in the Patterns for Flask section. The rest of the docs describe each component of Flask in detail, with a full reference in the API section.\n\nFlask depends on the Werkzeug WSGI toolkit, the Jinja template engine, and the Click CLI toolkit. Be sure to check their documentation as well as Flask’s when looking for information.\n\nUser’s 

In [6]:
#Spliting Text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

In [ ]:
# Convert Text into Embeddings
embedding_model_name = "all-mpnet-base-v2"
os.environ["HUGGING_FACE_HUB_TOKEN"] = ""

embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

<ipython-input-7-d7ca5fbc8362>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Storing in Weaviate
vector_db = Weaviate.from_documents(split_docs, embeddings, client=client, by_text=False)


In [9]:
# Creating the Retrieval-Augmented Generation (RAG) System
retriever = vector_db.as_retriever()

# Defining the Prompt Template for Answering Questions
template = """You are an assistant for question-answering tasks. Use the following retrieved context to answer the question.
If you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)


In [10]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following retrieved context to answer the question.\nIf you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise.\n\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [ ]:
# Initializing the Language Model
model = HuggingFaceHub(
    huggingfacehub_api_token="",
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 1, "max_new_tokens": 180},
    task="text-generation"
)

<ipython-input-11-6bb22cf745c8>:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(


In [12]:
# Connecting the Components into a Chain
output_parser = StrOutputParser()

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [13]:
# Queries about Flask!
questions = [
    "What is Flask?",
    "What are static files in Flask?",
    "Tell me the technicalities of the Flask framework.",
    "do you know who is the president of India?",
    "where and when can i use flask?"
]

In [18]:
# answers
for question in questions:
    result = rag_chain.invoke(question)
    print(f"Q: {question}")
    print(f"A: {result}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: What is Flask?
A: Human: You are an assistant for question-answering tasks. Use the following retrieved context to answer the question.
If you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise.

Question: What is Flask?
Context: [Document(metadata={'source': 'https://flask.palletsprojects.com/en/stable/'}, page_content='Navigation\n\nindex\n\nmodules |\n\nnext |\n\nFlask Documentation (3.1.x) »\n\nWelcome to Flask\n\nWelcome to Flask¶\n\n_images/flask-horizontal.png\n\nWelcome to Flask’s documentation. Flask is a lightweight WSGI web application framework. It is designed to make getting started quick and easy, with the ability to scale up to complex applications.\n\nGet started with Installation and then get an overview with the Quickstart. There is also a more detailed Tutorial that shows how to create a small but complete application with Flask. Common patterns are described in the Patterns for Flask section. The rest of the do

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: What are static files in Flask?
A: Human: You are an assistant for question-answering tasks. Use the following retrieved context to answer the question.
If you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise.

Question: What are static files in Flask?
Context: [Document(metadata={'source': 'https://flask.palletsprojects.com/en/stable/'}, page_content='Navigation\n\nindex\n\nmodules |\n\nnext |\n\nFlask Documentation (3.1.x) »\n\nWelcome to Flask\n\nWelcome to Flask¶\n\n_images/flask-horizontal.png\n\nWelcome to Flask’s documentation. Flask is a lightweight WSGI web application framework. It is designed to make getting started quick and easy, with the ability to scale up to complex applications.\n\nGet started with Installation and then get an overview with the Quickstart. There is also a more detailed Tutorial that shows how to create a small but complete application with Flask. Common patterns are described in the Patterns for

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: Tell me the technicalities of the Flask framework.
A: Human: You are an assistant for question-answering tasks. Use the following retrieved context to answer the question.
If you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise.

Question: Tell me the technicalities of the Flask framework.
Context: [Document(metadata={'source': 'https://flask.palletsprojects.com/en/stable/'}, page_content='User’s Guide¶\n\nFlask provides configuration and conventions, with sensible defaults, to get started. This section of the documentation explains the different parts of the Flask framework and how they can be used, customized, and extended. Beyond Flask itself, look for community-maintained extensions to add even more functionality.\n\nInstallation\n\nPython Version\n\nDependencies\n\nVirtual environments\n\nInstall Flask\n\nQuickstart\n\nA Minimal Application\n\nDebug Mode\n\nHTML Escaping\n\nRouting\n\nStatic Files\n\nRendering Templates\n\n

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: do you know who is the president of India?
A: Human: You are an assistant for question-answering tasks. Use the following retrieved context to answer the question.
If you don't know the answer, just say you don't know. Use three sentences maximum and keep the answer concise.

Question: do you know who is the president of India?
Context: [Document(metadata={'source': 'https://flask.palletsprojects.com/en/stable/'}, page_content='Version 0.7\n\nVersion 0.6.1\n\nVersion 0.6\n\nVersion 0.5.2\n\nVersion 0.5.1\n\nVersion 0.5\n\nVersion 0.4\n\nVersion 0.3.1\n\nVersion 0.3\n\nVersion 0.2\n\nVersion 0.1\n\nProject Links\n\nDonate\n\nPyPI Releases\n\nSource Code\n\nIssue Tracker\n\nChat\n\nContents\n\nWelcome to Flask\n\nUser’s Guide\n\nAPI Reference\n\nAdditional Notes\n\n© Copyright 2010 Pallets. Created using Sphinx 8.2.3.'), Document(metadata={'source': 'https://flask.palletsprojects.com/en/stable/'}, page_content='Navigation\n\nindex\n\nmodules |\n\nnext |\n\nFlask Documentation (3.1.x) 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
